# Хэш траблицы

## Введение

Хэш-таблица — это структура данных, которая реализует абстрактный тип данных "ассоциативный массив", позволяющий хранить пары (ключ, значение) и эффективно выполнять операции поиска, вставки и удаления.

## Таблицы с прямой адресацией

## Хеш-таблицы с разрешением коллизий с помощью цепочек

#### Теорема о сложности операций в хэш-таблице

**Теорема:** В хэш-таблице с методом разрешения коллизий через цепочки, при условии, что хэш-функция равномерно распределяет ключи, средняя сложность операций поиска, вставки и удаления составляет $O(1 + \alpha)$, где $\alpha = n/m$ — коэффициент заполнения таблицы, $n$ — количество элементов, $m$ — размер таблицы.

**Доказательство:**

Пусть $X_i$ — случайная величина, равная 1, если $i$-й элемент попадает в заданную ячейку, и 0 в противном случае. Тогда математическое ожидание числа элементов в ячейке:

$$E[X] = E[\sum_{i=1}^{n} X_i] = \sum_{i=1}^{n} E[X_i] = \sum_{i=1}^{n} \frac{1}{m} = \frac{n}{m} = \alpha$$

Таким образом, среднее число элементов в цепочке равно $\alpha$, и сложность операций составляет $O(1 + \alpha)$.


In [ ]:
# Реализация хэш-таблицы с методом цепочек
class HashTable:
    def __init__(self, size=10):
        """
        Инициализирует хэш-таблицу

        Параметры:
        size (int): Размер хэш-таблицы
        """
        self.size = size
        self.table = [[] for _ in range(size)]
        self.count = 0

    def _hash(self, key):
        """
        Вычисляет хэш-значение ключа

        Параметры:
        key: Ключ для хэширования

        Возвращает:
        int: Индекс в хэш-таблице
        """
        # Для строковых ключей
        if isinstance(key, str):
            hash_value = 0
            for char in key:
                hash_value = (hash_value * 31 + ord(char)) % self.size
            return hash_value
        # Для числовых ключей
        return key % self.size

    def insert(self, key, value):
        """
        Вставляет пару (ключ, значение) в хэш-таблицу

        Параметры:
        key: Ключ
        value: Значение
        """
        index = self._hash(key)

        # Проверяем, существует ли уже такой ключ
        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                self.table[index][i] = (key, value)
                return

        # Добавляем новую пару
        self.table[index].append((key, value))
        self.count += 1

        # Проверяем коэффициент заполнения
        if self.count / self.size > 0.7:
            self._resize(self.size * 2)

    def get(self, key):
        """
        Получает значение по ключу

        Параметры:
        key: Ключ

        Возвращает:
        value: Значение, соответствующее ключу, или None, если ключ не найден
        """
        index = self._hash(key)

        for k, v in self.table[index]:
            if k == key:
                return v

        return None

    def remove(self, key):
        """
        Удаляет пару по ключу

        Параметры:
        key: Ключ

        Возвращает:
        bool: True, если удаление успешно, False в противном случае
        """
        index = self._hash(key)

        for i, (k, v) in enumerate(self.table[index]):
            if k == key:
                del self.table[index][i]
                self.count -= 1
                return True

        return False

    def _resize(self, new_size):
        """
        Изменяет размер хэш-таблицы

        Параметры:
        new_size (int): Новый размер таблицы
        """
        old_table = self.table
        self.size = new_size
        self.table = [[] for _ in range(new_size)]
        self.count = 0

        # Перехэшируем все элементы
        for bucket in old_table:
            for key, value in bucket:
                self.insert(key, value)

    def load_factor(self):
        """
        Возвращает коэффициент заполнения таблицы

        Возвращает:
        float: Коэффициент заполнения
        """
        return self.count / self.size

    def __str__(self):
        """
        Строковое представление хэш-таблицы

        Возвращает:
        str: Строковое представление
        """
        result = "{\n"
        for i, bucket in enumerate(self.table):
            if bucket:
                result += f"  {i}: {bucket}\n"
        result += "}"
        return result

# Демонстрация работы хэш-таблицы
hash_table = HashTable(5)
hash_table.insert("один", 1)
hash_table.insert("два", 2)
hash_table.insert("три", 3)
hash_table.insert("четыре", 4)
hash_table.insert("пять", 5)

print("Хэш-таблица после вставки:")
print(hash_table)
print(f"Коэффициент заполнения: {hash_table.load_factor():.2f}")

print("\nПоиск значений:")
print(f"Значение для ключа 'два': {hash_table.get('два')}")
print(f"Значение для ключа 'шесть': {hash_table.get('шесть')}")

hash_table.remove("три")
print("\nХэш-таблица после удаления ключа 'три':")
print(hash_table)


## Хеш-таблицы с открытой адресацией

## Идеальное хеширование и static-hash

## Фильтры Блума

Фильтр Блума — это вероятностная структура данных, которая позволяет эффективно проверять принадлежность элемента множеству. Фильтр может давать ложноположительные срабатывания, но не ложноотрицательные.

#### Теорема о вероятности ложноположительного срабатывания

**Теорема:** Для фильтра Блума с $m$ битами, $k$ хэш-функциями и $n$ вставленными элементами, вероятность ложноположительного срабатывания составляет:

$$p \approx \left(1 - e^{-\frac{kn}{m}}\right)^k$$

**Доказательство:**

После вставки $n$ элементов, вероятность того, что конкретный бит остается равным 0, составляет:

$$p_0 = \left(1 - \frac{1}{m}\right)^{kn} \approx e^{-\frac{kn}{m}}$$

Следовательно, вероятность того, что бит равен 1, составляет:

$$p_1 = 1 - p_0 = 1 - e^{-\frac{kn}{m}}$$

Для ложноположительного срабатывания все $k$ битов, соответствующих хэш-значениям элемента, должны быть равны 1. Вероятность этого события:

$$p = p_1^k = \left(1 - e^{-\frac{kn}{m}}\right)^k$$

In [ ]:
# Реализация фильтра Блума
import math
import mmh3  # Библиотека MurmurHash3

class BloomFilter:
    def __init__(self, capacity, error_rate=0.01):
        """
        Инициализирует фильтр Блума

        Параметры:
        capacity (int): Ожидаемое количество элементов
        error_rate (float): Допустимая вероятность ложноположительного срабатывания
        """
        # Вычисляем оптимальные параметры
        self.size = self._optimal_size(capacity, error_rate)
        self.hash_count = self._optimal_hash_count(self.size, capacity)

        self.bit_array = [False] * self.size
        self.count = 0

    def _optimal_size(self, capacity, error_rate):
        """
        Вычисляет оптимальный размер битового массива

        Параметры:
        capacity (int): Ожидаемое количество элементов
        error_rate (float): Допустимая вероятность ложноположительного срабатывания

        Возвращает:
        int: Оптимальный размер битового массива
        """
        return int(-capacity * math.log(error_rate) / (math.log(2) ** 2))

    def _optimal_hash_count(self, size, capacity):
        """
        Вычисляет оптимальное количество хэш-функций

        Параметры:
        size (int): Размер битового массива
        capacity (int): Ожидаемое количество элементов

        Возвращает:
        int: Оптимальное количество хэш-функций
        """
        return max(1, int(size / capacity * math.log(2)))

    def add(self, item):
        """
        Добавляет элемент в фильтр

        Параметры:
        item: Элемент для добавления
        """
        for i in range(self.hash_count):
            index = mmh3.hash(str(item), i) % self.size
            self.bit_array[index] = True

        self.count += 1

    def contains(self, item):
        """
        Проверяет, содержится ли элемент в фильтре

        Параметры:
        item: Элемент для проверки

        Возвращает:
        bool: True, если элемент может содержаться в фильтре, False в противном случае
        """
        for i in range(self.hash_count):
            index = mmh3.hash(str(item), i) % self.size
            if not self.bit_array[index]:
                return False

        return True

    def false_positive_probability(self):
        """
        Вычисляет текущую вероятность ложноположительного срабатывания

        Возвращает:
        float: Вероятность ложноположительного срабатывания
        """
        return (1 - math.exp(-self.hash_count * self.count / self.size)) ** self.hash_count

    def __str__(self):
        """
        Строковое представление фильтра Блума

        Возвращает:
        str: Строковое представление
        """
        return f"BloomFilter(size={self.size}, hash_count={self.hash_count}, count={self.count}, fpp={self.false_positive_probability():.4f})"

# Демонстрация работы фильтра Блума
try:
    bloom = BloomFilter(1000, 0.01)

    # Добавляем элементы
    words = ["алгоритм", "структура", "данные", "хэш", "функция"]
    for word in words:
        bloom.add(word)

    print("Фильтр Блума:")
    print(bloom)

    # Проверяем наличие элементов
    test_words = ["алгоритм", "структура", "программирование", "анализ"]
    for word in test_words:
        result = bloom.contains(word)
        print(f"'{word}' {'содержится' if result else 'не содержится'} в фильтре")
        print(f"  (Фактически {'добавлен' if word in words else 'не добавлен'})")
except ImportError:
    print("Библиотека mmh3 не установлена. Установите ее с помощью 'pip install mmh3'")
    print("Для демонстрации используем простую реализацию фильтра Блума с базовыми хэш-функциями")

    # Простая реализация фильтра Блума без mmh3
    class SimpleBloomFilter:
        def __init__(self, size=1000, hash_count=3):
            self.size = size
            self.hash_count = hash_count
            self.bit_array = [False] * size
            self.count = 0

        def _hash1(self, item):
            """Первая хэш-функция"""
            h = 0
            for c in str(item):
                h = (h * 31 + ord(c)) % self.size
            return h

        def _hash2(self, item):
            """Вторая хэш-функция"""
            h = 0
            for c in str(item):
                h = (h * 37 + ord(c)) % self.size
            return h

        def add(self, item):
            """Добавляет элемент в фильтр"""
            for i in range(self.hash_count):
                # Используем линейную комбинацию двух хэш-функций
                index = (self._hash1(item) + i * self._hash2(item)) % self.size
                self.bit_array[index] = True
            self.count += 1

        def contains(self, item):
            """Проверяет, содержится ли элемент в фильтре"""
            for i in range(self.hash_count):
                index = (self._hash1(item) + i * self._hash2(item)) % self.size
                if not self.bit_array[index]:
                    return False
            return True

    # Демонстрация простого фильтра Блума
    bloom = SimpleBloomFilter(1000, 3)

    # Добавляем элементы
    words = ["алгоритм", "структура", "данные", "хэш", "функция"]
    for word in words:
        bloom.add(word)

    print("Простой фильтр Блума:")
    print(f"Размер: {bloom.size}, Количество хэш-функций: {bloom.hash_count}, Элементов: {bloom.count}")

    # Проверяем наличие элементов
    test_words = ["алгоритм", "структура", "программирование", "анализ"]
    for word in test_words:
        result = bloom.contains(word)
        print(f"'{word}' {'содержится' if result else 'не содержится'} в фильтре")
        print(f"  (Фактически {'добавлен' if word in words else 'не добавлен'})")


## Алгоритм min-hash